Clean up data:
1. Remove the repeated heading
2. Remove the repeated summations
3. Separate game name and creator

In [1]:
from csv import reader
import pandas as pd

open_file = open('my_data.csv', encoding = 'utf-8')
read_file = reader(open_file)
rotrends = list(read_file)
rotrends_header = rotrends[0]
rotrends = rotrends[1:]

In [2]:
def find_duples_uniq(dataset):
    duples = []
    uniq = []
    for app in dataset:
        name = app[0]
        if name in uniq:
            duples.append(name)
        else:
            uniq.append(name)

    return duples, uniq
duples_uniq_rotrends = find_duples_uniq(rotrends) 
print(len(duples_uniq_rotrends[0]))
print(len(duples_uniq_rotrends[1]))

270
8093


In [3]:
def remove_duples(dataset):
    rank = {}
    uniq_app = []
    already_add=[]
    for app in dataset:
        name = app[0]
        game_rank = app[1]
        if name in rank and rank[name] < game_rank:
            rank[name] = game_rank
        elif name not in rank:
            rank[name] = game_rank
    for app in dataset:
        name = app[0]
        game_rank = app[1]
        if (rank[name] == game_rank) and (name not in already_add):
            uniq_app.append(app)
            already_add.append(name)
    return uniq_app
        

In [4]:
uniq_app_rotrends = remove_duples(rotrends)

In [5]:
from datetime import datetime
date_format = "%m/%d/%Y"
def is_valid_date(date_string, date_format):
    
        try:
            datetime.strptime(date_string, date_format)
            return True
        except ValueError:
            return False

def remove_headers_sums(dataset):
    rotrends_apps = []
    for app in dataset:
        date_created = app[15]
        if is_valid_date(date_created,date_format):
            rotrends_apps.append(app)

    return rotrends_apps

In [6]:
almost_clean = remove_headers_sums(uniq_app_rotrends)
print(len(almost_clean))

8090


In [7]:
def split_name_dev(dataset):
    name =[]
    dev_name = []
    for app in dataset:
        game_name = app[0]
        i = game_name.find('@')
        name.append(game_name[:i])
        dev_name.append(game_name[i+1:])
    return [name, dev_name]

In [8]:
print(rotrends_header)
print(almost_clean[0])

['Games', 'Global Rank', 'Global Rank Shift (1w)', 'Global Rank Shift (1m)', 'Avg CCU Rank Shift (1d)', 'Earning Rank', 'Genre', 'Sub Genre', 'Visits', 'Players (CCU)', 'Avg CCU (7d)', 'Avg CCU (14d)', 'Momentum (1w)', 'Momentum (1m)', 'Avg Session', 'Created']
['[💎UPD] Fish It! 🐟@Fish Atelier', '#21', '+1', '+3', '0', '#6', 'Simulation', 'Incremental Simulator', '1.63B', '1,017,825', '1,025,642', '968,402', 'Rising', 'Surging', '36.5', '10/20/2024']


In [9]:
rotrends_data = pd.DataFrame(almost_clean, columns = rotrends_header)
df = rotrends_data
# Split the 'Name' column into 'First_Name' and 'Last_Name' using a space as a delimiter
df[['Games', 'Dev_Name']] = df['Games'].str.split('@', n = 1, expand=True)

In [15]:
import requests as re

df["Global Rank"]  = df["Global Rank"].str.extract(r'(\d*\.\d+|\d+)', expand=False)
df["Earning Rank"]  = df["Earning Rank"].str.extract(r'(\d*\.\d+|\d+)', expand=False)

In [16]:
# Export DataFrame 
df.to_csv('output.csv', header=True, index=False)

print("Data exported to 'output.csv' with header.")

Data exported to 'output.csv' with header.
